In [1]:
from db_communication import db_communication
import numpy as np
import pandas as pd
from datetime import datetime

### Init tables/ dataframes

In [2]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'Fahrradshop' 
}
my_db = db_communication(config)

Successfully connected.


In [3]:
cluster_df = my_db.get_table('Merkmalcluster')
parts_df = my_db.get_table('Einzelteile')
features_df = my_db.get_table('Merkmale')

config_df = my_db.get_table('Konfiguration')
config_df.index = pd.MultiIndex.from_arrays([config_df['AuftragNr'],config_df['KonfigNr']])
config_df = config_df['MerkmalNr'].to_frame()

assemblyGroup_df = my_db.get_table("Arbeitsschrittgruppe")

### Direct demand matrix by database content

In [4]:
# generate Multiindex for the columns
col_main = cluster_df['Merkmalsbezeichnung'][features_df['Merkmalcluster'].values-1]
col_main.name = 'Cluster'
col_sub = features_df['Bezeichnung']
col_sub.name = 'Merkmal'
col = pd.MultiIndex.from_arrays([col_main,col_sub])

In [5]:
# part name as index 
index_names = parts_df['Bezeichnung']
index_names.name = 'Einzelteil'

In [6]:
# fill data with NaNs
data = np.ones((len(index_names),len(col))) * np.nan

df_temp = pd.DataFrame(data=data,columns=col,index=index_names)
df_temp

Cluster                                          Rahmen                      \
Merkmal                                 Herren Citybike Herren Trekkingbike   
Einzelteil                                                                    
Surly Long Haul Trucker Rahmenkit                   NaN                 NaN   
Surly Troll Rahmenkit                               NaN                 NaN   
Salsa Cutthroat Rahmenkit                           NaN                 NaN   
VOTEC VRC Framekit                                  NaN                 NaN   
Cervelo C3 Frameset                                 NaN                 NaN   
...                                                 ...                 ...   
Xpedo Detox Pedale                                  NaN                 NaN   
NOW8 M36 Flat Pedals 6 Pins                         NaN                 NaN   
Moto Reflex Pedale                                  NaN                 NaN   
Time ATAC XC6 X-Country Pedale                      NaN                 NaN   
Shimano Dura Ace PD-R9100 Pedale carbon             NaN                 NaN   

Cluster                                                         \
Merkmal                                 Herren Trekking Carbon   
Einzelteil                                                       
Surly Long Haul Trucker Rahmenkit                          NaN   
Surly Troll Rahmenkit                                      NaN   
Salsa Cutthroat Rahmenkit                                  NaN   
VOTEC VRC Framekit                                         NaN   
Cervelo C3 Frameset                                        NaN   
...                                                        ...   
Xpedo Detox Pedale                                         NaN   
NOW8 M36 Flat Pedals 6 Pins                                NaN   
Moto Reflex Pedale                                         NaN   
Time ATAC XC6 X-Country Pedale                             NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                    NaN   

Cluster                                                           \
Merkmal                                 Unisex Rennrad Aluminium   
Einzelteil                                                         
Surly Long Haul Trucker Rahmenkit                            NaN   
Surly Troll Rahmenkit                                        NaN   
Salsa Cutthroat Rahmenkit                                    NaN   
VOTEC VRC Framekit                                           NaN   
Cervelo C3 Frameset                                          NaN   
...                                                          ...   
Xpedo Detox Pedale                                           NaN   
NOW8 M36 Flat Pedals 6 Pins                                  NaN   
Moto Reflex Pedale                                           NaN   
Time ATAC XC6 X-Country Pedale                               NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                      NaN   

Cluster                                                                       \
Merkmal                                 Unisex Rennrad Carbon Damen Citybike   
Einzelteil                                                                     
Surly Long Haul Trucker Rahmenkit                         NaN            NaN   
Surly Troll Rahmenkit                                     NaN            NaN   
Salsa Cutthroat Rahmenkit                                 NaN            NaN   
VOTEC VRC Framekit                                        NaN            NaN   
Cervelo C3 Frameset                                       NaN            NaN   
...                                                       ...            ...   
Xpedo Detox Pedale                                        NaN            NaN   
NOW8 M36 Flat Pedals 6 Pins                               NaN            NaN   
Moto Reflex Pedale                                        NaN            NaN   
Time ATAC XC6 X-Country Pedale                         

In [7]:
def highlight_newEntries(df_temp, compare_filename):
    """ Compare and with an older version of df_temp and highlight new rows and columns """
    # read old df for comparisions
    compare_df = pd.read_excel(compare_filename, header=[0,1], index_col=0)

    # determine new columns and rows
    new_rows = [row for row in df_temp.index.values if row not in compare_df.index.values]
    new_cols = [col for col in df_temp.columns if col not in compare_df.columns]

    # highlight these rows/columns
    def highlight_new_rowsAndCols(s):
        col_name = s.name
        if col_name in new_cols:
            return ['background-color: yellow' for i in range(len(s))]
        else:
            return ['' if (row not in new_rows) else 'background-color: yellow' for row in s.index]
    df_temp = df_temp.style.apply(highlight_new_rowsAndCols)
    return df_temp

In [8]:
def save_directDemandMatrix(df_temp, compare_filename=None):
    """ 
    save df_temp as direct demand matrix 
    Compare and with an older version of df_temp and highlight new rows and columns whether compare_filename exists
    """
    # if 'compare_filename' exists then highlight new rows and columns in 'df_temp'
    if type(compare_filename) == str:
        df_temp = highlight_newEntries(df_temp,compare_filename)
    
    # save template as Excel with current datetime as version
    new_filename = 'DirectDemandMatrix_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx'
    df_temp.to_excel(new_filename)
    return new_filename

In [ ]:
save_directDemandMatrix(df_temp,compare_filename='DirectDemandMatrix_20200623_165446.xlsx')

### Compare new and old versions of direct demand data frame

In [9]:
# read excel
filename = 'DirectDemandMatrix_20200623_165446.xlsx'
directDemand_df = pd.read_excel(filename, header=[0,1], index_col=0)

In [10]:
#compare rows
delta_n_rows = len(df_temp) - len(directDemand_df)
new_rows = [row for row in df_temp.index.values if row not in directDemand_df.index.values]
delta_n_rows, new_rows

(0, [])

In [11]:
#compare columns
delta_n_cols = len(df_temp.columns) - len(directDemand_df.columns)
new_cols = [col for col in df_temp.columns if col not in directDemand_df.columns]
delta_n_cols, new_cols

(0, [])

### Verify direct demand matrix

In [12]:
directDemand_df

Cluster                                          Rahmen                      \
Merkmal                                 Herren Citybike Herren Trekkingbike   
Einzelteil                                                                    
Surly Long Haul Trucker Rahmenkit                   1.0                 NaN   
Surly Troll Rahmenkit                               NaN                 1.0   
Salsa Cutthroat Rahmenkit                           NaN                 NaN   
VOTEC VRC Framekit                                  NaN                 NaN   
Cervelo C3 Frameset                                 NaN                 NaN   
...                                                 ...                 ...   
Xpedo Detox Pedale                                  NaN                 NaN   
NOW8 M36 Flat Pedals 6 Pins                         NaN                 NaN   
Moto Reflex Pedale                                  NaN                 NaN   
Time ATAC XC6 X-Country Pedale                      NaN                 NaN   
Shimano Dura Ace PD-R9100 Pedale carbon             NaN                 NaN   

Cluster                                                         \
Merkmal                                 Herren Trekking Carbon   
Einzelteil                                                       
Surly Long Haul Trucker Rahmenkit                          NaN   
Surly Troll Rahmenkit                                      NaN   
Salsa Cutthroat Rahmenkit                                  1.0   
VOTEC VRC Framekit                                         NaN   
Cervelo C3 Frameset                                        NaN   
...                                                        ...   
Xpedo Detox Pedale                                         NaN   
NOW8 M36 Flat Pedals 6 Pins                                NaN   
Moto Reflex Pedale                                         NaN   
Time ATAC XC6 X-Country Pedale                             NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                    NaN   

Cluster                                                           \
Merkmal                                 Unisex Rennrad Aluminium   
Einzelteil                                                         
Surly Long Haul Trucker Rahmenkit                            NaN   
Surly Troll Rahmenkit                                        NaN   
Salsa Cutthroat Rahmenkit                                    NaN   
VOTEC VRC Framekit                                           1.0   
Cervelo C3 Frameset                                          NaN   
...                                                          ...   
Xpedo Detox Pedale                                           NaN   
NOW8 M36 Flat Pedals 6 Pins                                  NaN   
Moto Reflex Pedale                                           NaN   
Time ATAC XC6 X-Country Pedale                               NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                      NaN   

Cluster                                                                       \
Merkmal                                 Unisex Rennrad Carbon Damen Citybike   
Einzelteil                                                                     
Surly Long Haul Trucker Rahmenkit                         NaN            NaN   
Surly Troll Rahmenkit                                     NaN            NaN   
Salsa Cutthroat Rahmenkit                                 NaN            NaN   
VOTEC VRC Framekit                                        NaN            NaN   
Cervelo C3 Frameset                                       1.0            NaN   
...                                                       ...            ...   
Xpedo Detox Pedale                                        NaN            NaN   
NOW8 M36 Flat Pedals 6 Pins                               NaN            NaN   
Moto Reflex Pedale                                        NaN            NaN   
Time ATAC XC6 X-Country Pedale                         

In [13]:
# number of all combinations
directDemand_df.count(axis=1,numeric_only=True).sum()

218

In [14]:
# difference to the generated template
missing_cols = [col for col in df_temp.columns if col not in directDemand_df.columns]
missing_rows = [row for row in df_temp.index if row not in directDemand_df.index]
missing_cols, missing_rows

([], [])

In [15]:
# check columns
print("Columns with NaNs only:\n")
for col_name,col in directDemand_df.iteritems():
    if col.isnull().all():
        print(col_name)

Columns with NaNs only:



In [16]:
# check rows
print("Rows with NaNs only:\n")
for row_name,row in directDemand_df.iterrows():
    if row.isnull().all():
        print(row_name)

Rows with NaNs only:

Shimano Alfine BL-S7000 Scheibenbremse I-Spec II Hinten


__validate the direct demand matrix by some configuration examples:__

In [17]:
def checkConfig(orderId,
                features_df=features_df,cluster_df=cluster_df,
                config_df=config_df, directDemand_df=directDemand_df):
    """
    verification through test configurations
    """
    # init lists
    feature_names = []
    part_names = []
    part_count = []
    for row_name,row in config_df.loc[orderId].iterrows():
        # map feature id to feature name and cluster
        feature_i = features_df.set_index('MerkmalNr').loc[row.values[0]]
        feature_name = feature_i['Bezeichnung']
        # get current cluster name
        feature_cluster = feature_i['Merkmalcluster']
        cluster_name = cluster_df.set_index('ClusterNr').loc[str(feature_cluster)].iloc[0]

        # get current parts per feature
        parts = directDemand_df[cluster_name,feature_name].dropna() # <-- crucial step
        if len(parts) != 0:
            part_names.append(parts.index.to_list())
        else:
            part_names.append([None])
        part_count.append(parts.values.astype(int))
        feature_names.append([feature_name for i in range(len(part_names[-1]))])

    # flatten lists for multiindexing
    feature_names = [feature_i for feature_name in feature_names for feature_i in feature_name] 
    part_names = [part_i for part_name in part_names for part_i in part_name] 

    # to df
    index = pd.MultiIndex.from_arrays([feature_names,part_names])
    index.names=['Merkmal','Einzelteil']
    df = pd.DataFrame(index=index,columns=['Anzahl'])
    # fill data
    i = 0
    for part_i in part_count:
        if len(part_i) != 0:
            for j in part_i:
                df.iloc[i] = [j]
                i += 1
        else:
            df.iloc[i] = [None]
            i += 1

    return df

In [18]:
checkConfig(5)

Anzahl
Merkmal                      Einzelteil                                               
Rennrad Junior               Giant Trinity Advanced Pro Time Trial                   1
                             Kein Fahradstaender                                     1
                             Kein Gepaecktraeger                                     1
                             Kein Schutzblech                                        1
Federung Standard            SR Suntour SF18 Nex-E25 HLO P Federgabel                1
Blau metallic                Blau Metallic Basislack                                 1
                             2K HS Fueller hellgrau                                  1
                             2K HS Haerter fur Fueller                               1
                             Silikonentferner                                        1
Lenker Standard mit Hoernern Humpert Cruiser Lenker                                  1
                             SGODDE Fahrradklingel                                   1
                             Ergon GP3 Griffe Gripshift                              2
Rennradfelgen Pro            Mavic Comete Pro Carbon SL T Laufradsatz Shiman...      2
                             Shimano Nexus DH-C3000-3N Nabendynamo                   1
Bereifung Sport              SCHWALBE HS 180 Drahtreifen 26 Zoll Active Line...      2
                             WTB Presta Tubeless Ventil Aluminium 46mm black         2
                             SCHWALBE No.13 Fahrradschlauch                          2
Rennradschaltung             Shimano FC-R7000 Kurbelgarnitur                         1
                             Shimano ST-R7025 Schaltgriff                            1
                             Shimano RD-R7000 GS Schaltwerk                          1
                             Shimano CN-HG601 Fahrrad-Kette                          1
                             Shimano CS-R7000 Fahrrad-Kassette                       1
                             Shimano SM-BBR60 Innenlager                             1
                             Kein Kettenschutz                                       1
Rennradsattel Herren         SQlab 612 Ergowave Sattel S-Tube                        1
Scheibenbremse Standard      Shimano BL-T4000 Bremshebel                             1
                             Shimano BR-R7000 Felgenbremse Dual-Pivot Hinter...      1
                             Jagwire Mountain Sport                                  2
                             Shimano Alfine BL-S7000 Scheibenbremse I-Spec I...      1
                             Shimano Deore SM-RT56 Bremsscheibe 6-loch               1
                             Shimano BH90-JK-SSR                                     1
Beleuchtung Pro              MonkeyLink MonkeyLight 70 Lux Recharge Vorne            1
                             MonkeyLink MonkeyLight 70 Lux Recharge Hinten           1
                             Dynamo RECHTS 6V/3W Überspannungsschutz Doppela...      1
                             Leuchtstreifen                                          2
Trekking Pedale              Xpedo Detox Pedale                                      2